Перейдём к 4ой модели случайному лесу, потихоньку надежда на ML пропадает, как работает берётся куча деревьев решений и через них прогоняются, дале мы принимаем решения либо усредняя либ по выбору большинства

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import accuracy_score, f1_score
import time
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
split_idx = int(0.8 * len(train))
test = train.iloc[split_idx:]
train = train.iloc[:split_idx]

X_train = train.drop('Fertilizer Name', axis=1)
y_train = train['Fertilizer Name']

X_test = test.drop('Fertilizer Name', axis=1)
y_test = test['Fertilizer Name']


from sklearn.preprocessing import OneHotEncoder
cat_cols = ['Soil Type', 'Crop Type']
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[cat_cols]).toarray()
feature_names = encoder.get_feature_names_out(cat_cols)
X_test_encoded = encoder.transform(X_test[cat_cols]).toarray()

from sklearn.preprocessing import StandardScaler
num_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num_cols])
X_test_scaled = scaler.transform(X_test[num_cols])

X_train_processed = np.hstack([X_train_scaled, X_train_encoded])
X_test_processed = np.hstack([X_test_scaled, X_test_encoded])

results = {}

In [10]:
print("\nОбучение Random Forest...")
start_time = time.time()
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_processed, y_train)
y_pred = rf.predict(X_test_processed)
results["Random Forest"] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred, average='weighted')
}
print(f"Завершено за {time.time() - start_time:.2f} сек. F1: {results['Random Forest']['F1 Score']:.4f}")


Обучение Random Forest...
Завершено за 133.48 сек. F1: 0.1632


Как мы видим модель обучается не быстро так что применим уже нам знакомый RandomizedSearchCV для ускорения обучения.

Обсудим за что ответсвенен каждый гиперпараметр, который мы перебираем

n_estimators-Количество деревьев

max_depth-Макс. глубина: контроль сложности модели

min_samples_split - Мин. образцов для разделения: предотвращение переобучения

max_features - Количество признаков для разделения

In [13]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import time


param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2']
}


start_time = time.time()

rf_base = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,
    n_jobs=-1
)
grid_search.fit(X_train_processed, y_train)

print(f"Подбор завершен за {time.time() - start_time:.2f} сек")
print(f"Лучшие параметры: {grid_search.best_params_}")

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_processed)

results["Random Forest (optimized)"] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, average='weighted'),
    'Recall': recall_score(y_test, y_pred, average='weighted'),
    'F1 Score': f1_score(y_test, y_pred, average='weighted')
}

for metric, value in results["Random Forest (optimized)"].items():
    print(f"{metric}: {value:.4f}")

Подбор завершен за 1294.23 сек
Лучшие параметры: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 150}
Accuracy: 0.1775
Precision: 0.1760
Recall: 0.1775
F1 Score: 0.1689
